In [ ]:
# email_assistant.py
#
# Demo of LLMs + Prompt Engineering
# 1. Summarise long customer emails
# 2. Expand internal summaries into polished customer emails
#
# Prereqs:
# pip install openai langchain python-dotenv
# export OPENAI_API_KEY="sk-..."
# or use dotenv to load from .env

import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

load_dotenv()
llm = ChatOpenAI(model="gpt-4", temperature=0.25)

# Prompts
summary_prompt = PromptTemplate.from_template("""
You are an efficient customer support assistant.
Summarise this customer email in 1-2 sentences for an internal note.

Email:
{email}
""")

expand_prompt = PromptTemplate.from_template("""
You are a customer service agent. Turn this internal summary
into a full reply in {tone} tone. Keep it under 150 words and sign off as 'Support Team'.

Summary:
{summary}
""")


def summarise_email(email: str) -> str:
    """Summarises a long email to a short internal note."""
    prompt = summary_prompt.format(email=email)
    response = llm.predict(prompt)
    return response.strip()


def expand_summary(summary: str, tone: str = "friendly") -> str:
    """Expands an internal summary to a full email in the specified tone."""
    prompt = expand_prompt.format(summary=summary, tone=tone)
    response = llm.predict(prompt)
    return response.strip()

if __name__ == "__main__":
    # Demo email from a customer
    email_text = """
    Hello, I ordered a garden table over two weeks ago and it still hasn't arrived.
    Your site said delivery would be within 5 days. I’ve tried calling twice but no one picks up.
    I need this urgently for a party this weekend. Please respond ASAP or I’ll need to cancel and get a refund.
    """

    print("\n--- CUSTOMER EMAIL ---\n", email_text.strip())

    summary = summarise_email(email_text)
    print("\n--- INTERNAL SUMMARY ---\n", summary)

    reply = expand_summary(summary, tone="apologetic")
    print("\n--- POLISHED REPLY (Apologetic Tone) ---\n", reply)


In [ ]:
reply = expand_summary(summary, tone="friendly")
print("\n--- POLISHED REPLY (Friendly Tone) ---\n", reply)

In [ ]:
reply = expand_summary(summary, tone="joking")
print("\n--- POLISHED REPLY (Joking Tone) ---\n", reply)

In [ ]:
reply = expand_summary(summary, tone="rude and obnoxious")
print("\n--- POLISHED REPLY (Rude Tone) ---\n", reply)

In [ ]:
reply = expand_summary(summary, tone="horrified and overly dramatic")
print("\n--- POLISHED REPLY (Dramatic Tone) ---\n", reply)

In [ ]:
# meeting_assistant.py
#
# Demo: LangChain tooling with modern prompt chaining
# 1. Extract action items from a meeting transcript
# 2. Generate a follow-up email
#
# Prereqs:
# pip install langchain langchain-openai langchain-community openai python-dotenv

import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import SystemMessage, HumanMessage

load_dotenv()
llm = ChatOpenAI(model="gpt-4", temperature=0.2)

# ────────────────────────────────────────────────────────────────────────────────
# ACTION ITEM EXTRACTOR
# ────────────────────────────────────────────────────────────────────────────────

action_item_prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a meeting assistant. Extract clear, concise action items from the transcript."),
    HumanMessage(content="{transcript}")
])

action_chain = action_item_prompt | llm

def extract_action_items(transcript: str) -> str:
    return action_chain.invoke({"transcript": transcript}).content.strip()

# ────────────────────────────────────────────────────────────────────────────────
# FOLLOW-UP EMAIL GENERATOR
# ────────────────────────────────────────────────────────────────────────────────

email_prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a helpful team coordinator. Write a short email summarising the agreed actions."),
    HumanMessage(content="Action items:\n{items}")
])

email_chain = email_prompt | llm

def generate_follow_up_email(items: str) -> str:
    return email_chain.invoke({"items": items}).content.strip()

# ────────────────────────────────────────────────────────────────────────────────
# DEMO
# ────────────────────────────────────────────────────────────────────────────────

if __name__ == "__main__":
    raw_transcript = """
    Okay so first, Sarah is going to finalise the budget proposal by Friday.
    Mike said he’d handle reaching out to the vendors about new pricing.
    I’ll update the project timeline and send the revised version by end of day Thursday.
    We agreed to do another check-in next Tuesday at 3pm.
    """

    print("\n--- RAW TRANSCRIPT ---\n", raw_transcript.strip())

    actions = extract_action_items(raw_transcript)
    print("\n--- EXTRACTED ACTION ITEMS ---\n", actions)

    email = generate_follow_up_email(actions)
    print("\n--- FOLLOW-UP EMAIL ---\n", email)


In [ ]:
# meeting_assistant.py
#
# Demo of LLMs + Prompt Engineering
# 1. Summarise long customer emails
# 2. Expand internal summaries into polished customer emails
#
# Prereqs:
# pip install openai langchain python-dotenv
# export OPENAI_API_KEY="sk-..."
# or use dotenv to load from .env

import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

load_dotenv()
llm = ChatOpenAI(model="gpt-4", temperature=0.25)

# Prompts
summary_prompt = PromptTemplate.from_template("""
You are a meeting assistant. Extract clear, concise action items from the transcript.

Transcript:
{transcript}
""")

email_prompt = PromptTemplate.from_template("""
You are a helpful team coordinator. Write a short email summarising the agreed actions.
Sign off as Team Coordinator bot. Be super friendly, optimistic and motivating.

Action items:
{items}
""")


def extract_action_items(transcript: str) -> str:
    """Summarises a long transcript to a short internal items."""
    prompt = summary_prompt.format(transcript=transcript)
    response = llm.invoke(prompt)
    return response.content.strip()


def generate_follow_up_email(items: str) -> str:
    """Send an email describing the actions."""
    prompt = email_prompt.format(items=items)
    response = llm.invoke(prompt)
    return response.content.strip()

if __name__ == "__main__":
    raw_transcript = """
    How was yopur weekend Dave?
    <missed>
    Yeah mine was fine.
    Okay so first, Sarah is going to finalise the budget proposal by Friday.
    Mike said he’d handle reaching out to the vendors about new pricing.
    I’ll update the project timeline and send the revised version by end of day Thursday.
    We agreed to do another check-in next Tuesday at 3pm.
    """

    print("\n--- RAW TRANSCRIPT ---\n", raw_transcript.strip())

    actions = extract_action_items(raw_transcript)
    print("\n--- EXTRACTED ACTION ITEMS ---\n", actions)

    email = generate_follow_up_email(actions)
    print("\n--- FOLLOW-UP EMAIL ---\n", email)
